In [7]:
import cv2
import os
from  skimage.io import imread, imshow
from skimage import transform
import pywt
import pywt.data
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Encontrando los bordes de las imagenes

In [39]:
path_segmentacion = 'mariposas/segmentations'
output_file = "mariposas.csv"

# Abre el archivo TXT en modo de escritura
with open(output_file, 'w') as file:
    file.write(f"Width,Height,Roi.X1,Roi.Y1,Roi.X2,Roi.Y2,ClassId,Path\n")
    # Itera a través de las imágenes y obtiene las características
    for image_name in os.listdir(path_segmentacion):
        image_path = os.path.join(path_segmentacion, image_name)
        # Cargar la imagen segmentada
        imagen_segmentada = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        # Aplicar umbralización
        _, imagen_umbralizada = cv2.threshold(imagen_segmentada, 128, 255, cv2.THRESH_BINARY)
        # Encontrar contornos
        contornos, _ = cv2.findContours(imagen_umbralizada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Calcular coordenadas (x1, x2, y1, y2)
        x, y, w, h = cv2.boundingRect(contornos[0])
        x1, x2, y1, y2 = x, x + w, y, y + h
        # Formatea los datos como una cadena y escribe en el archivo TXT
        alto, ancho = imagen_segmentada.shape
        classid = int(image_name[:3])
        data_line = f"{ancho},{alto},{x1},{y1},{x2},{y2},{classid},{image_name[:7]}.png\n"
        file.write(data_line)
print(f"Los datos se han guardado en {output_file}")

Los datos se han guardado en mariposas.csv


In [3]:
def Get_Feacture(picture, cortes = 3):
   LL = picture
   for i in range(cortes):
      LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
   return LL.flatten()

# Vectores caracteristicos para las imagenes sin fondo

In [23]:
path_csv = 'mariposas.csv'
print(f'Loading {path_csv}')
index = pd.read_csv(path_csv)
index_records = index[['Roi.X1', 'Roi.X2', 'Roi.Y1', 'Roi.Y2', 'ClassId', 'Path']]
encodings, classes = list(), list()
for _, (x1, x2, y1, y2, class_id, path_name_img) in index_records.iterrows():
    img = imread(f'mariposas/images/{path_name_img}')[y1:y2, x1:x2]
    img_res = transform.resize(img, (250, 250, 3))
    img_encoding = Get_Feacture(img_res, 4)
    encodings.append(img_encoding)
    classes.append(class_id)
print(f'Loaded {len(encodings)} records')
X_all = np.array(encodings)
Y_all = np.array(classes)
#print(X_all, Y_all)

Loading mariposas.csv
Loaded 832 records


In [24]:
# Crear un DataFrame de Pandas con los datos
data = {'X': X_all.tolist(), 'Y': Y_all.tolist()}
df = pd.DataFrame(data)
output_csv = 'vc_mariposas_sin_fondo_250_4.csv'
df.to_csv(output_csv, index=False)
print(f'Datos guardados en {output_csv}')


Datos guardados en vc_mariposas_sin_fondo_250_4.csv


# Vectores caracteristicos para las imagenes con fondo

In [20]:
path_mariposas = 'mariposas/images'
print(f'Loading {path_mariposas}')
encodings, classes = list(), list()
for image_name in os.listdir(path_mariposas):
    image_path = os.path.join(path_mariposas, image_name)
    img = imread(image_path)
    img_res = transform.resize(img, (250, 250, 3))
    img_encoding = Get_Feacture(img_res, 4)
    encodings.append(img_encoding)
    classes.append(int(image_name[:3]))
print(f'Loaded {len(encodings)} records')
X_all = np.array(encodings)
Y_all = np.array(classes)

Loading mariposas/images
Loaded 832 records


In [22]:
# Crear un DataFrame de Pandas con los datos
data = {'X': X_all.tolist(), 'Y': Y_all.tolist()}
df = pd.DataFrame(data)
output_csv = 'vc_mariposas_original_250_4.csv'
df.to_csv(output_csv, index=False)
print(f'Datos guardados en {output_csv}')

Datos guardados en vc_mariposas_original_250_4.csv
